In [90]:
import pandas as pd
df_2021 = pd.read_csv("2021.csv");
df_2020 = pd.read_csv("2020.csv");
df_2019 = pd.read_csv("2019.csv");
df_2018 = pd.read_csv("2018.csv");

df = pd.concat([df_2018, df_2019, df_2020, df_2021], ignore_index=True);


df.Timestamp = pd.to_datetime(df.Timestamp)
df.Quantity = df.Quantity.str.replace(",", "")
df.Quantity = pd.to_numeric(df.Quantity)
df["Trade Type"].unique()

array(['Buy', 'Sell', 'Mining', 'Transfer Out', 'Fee',
       'Unknown Acquisition', 'Transfer In', 'Airdrop', 'Failed Out',
       'Staking', 'Borrow', 'Loan Repayment', 'Staking Deposit',
       'Staking Withdrawal', 'Burn', 'Mint', 'Interest', 'Income',
       'Fiat Deposit', 'Cashback', 'Unknown Disposal', 'Fiat Withdrawal',
       'Ignore In', 'Royalties', 'Unknown'], dtype=object)

In [91]:
# numeric_columns = ["Quantity", "Opening Cost", "Opening Value", "Ending Cost", "Closing Value", "Opening Balance", "Closing Balance"]

# for x in numeric_columns:
#     df[x] = pd.to_numeric(df[x].str.replace(",","").replace("^-$", "0", regex=True))


In [92]:
#get all trade types
df["Trade Type"].unique()
income_types = ["Income", "Airdrop"]
income_transactions = df[df["Trade Type"].isin(income_types) ]

tax_types = ["Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In"]
#remove all income transactions from tax calculations
tax_df = df[~df["Trade Type"].isin(income_types)]


income_types = ["Income", "Airdrop"]
income_tx = df[df["Trade Type"].isin(income_types) ]

non_taxed_types = ["Cashback", "Borrow", "Unknown Acquisition", "Mining"]
non_tax_df = df[df["Trade Type"].isin(non_taxed_types)]

staking_types =["Staking", "Staking Withdrawal"]
staked_df = df[df["Trade Type"].isin(staking_types)]

In [93]:
# store all income tokens seperately
all_coins = df.Currency.unique()
income_tokens = {}
for token_name in all_coins:
    this_income_tokens = tax_df[tax_df.Currency == token_name ]
    income_tokens[token_name] = this_income_tokens

In [94]:
#create a column to store the ACB at each sell transactions
tax_df["ACB"]  = None;
#tax_df.apply(lambda x: print (x.name), axis=1)


def calc_acb(prev_acb, row):
    result = {"cost": 0,  "proceed": 0, "fee": 0}

    #read the last acb
    if (prev_acb is not None):
        this_acb = prev_acb.copy() 
    else:
        this_acb = pd.DataFrame(columns = ["Currency", "Quantity", "Price", "Trade Type"])
        row["ACB"] = this_acb
#tax_types = ["Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In"]
 
    trade_type = row["Trade Type"]

    if (trade_type in ["Sell","Fee"] ):
        
        #selling, 
        sell_quantity = row["Quantity"]
        left_over_sell_quantity = sell_quantity
        rows_to_delete = [];
        curr_cost = 0;
        for _, acb_row in this_acb.iterrows():
            #for each row, deduct the amount to be sold 
            delta = max(sell_quantity, acb_row.Quantity)
            left_over_sell_quantity -= delta
            

            # if (left_over_sell_quantity < 0) :
            #     #for now, we do not handle negative amount
            #     raise "Negetive amount"

            acb_type = acb_row["Trade Type"]

            if (acb_type in ["Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In"]):
                # if the acb type is already taxed, dont tax again, so use today's price to calc cost so that no gain can incur
                acb_cost = row.Price * delta
                curr_cost += acb_cost
            else:
                #otherwise the cost of the acb entry will be the actual cost
                curr_cost += acb_row.Price * delta

            if (left_over_sell_quantity > 0):
                #if there are more to sell remove this row
                acb_row.Quantity -= delta
            else:
                #other wise adjust the number of this row, and we are done
                rows_to_delete.append(acb_row.name)
                break;

        result["cost"] = curr_cost
        result["proceed"] = sell_quantity * row.Price

        gains = result["proceed"] - result["cost"]
        if (trade_type == "Fees"):
            fee = sell_quantity * row.Price
            result["fee"] = fee
            print(f"Paid Fee {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp} fee ${fee} gained {gains}");
        else:
            print(f"Selling {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp} gained {gains}");

        this_acb.drop(index = rows_to_delete , inplace=True)
            #otherwise go on to the next row
    elif (trade_type in ["Interest", "Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In", "Buy"]):
        #add to the acb
        print(f"{trade_type} {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp}");
        this_acb.loc[len(this_acb.index)] = [row.Currency, row.Quantity, row.Price, trade_type]
    elif(trade_type in ["Transfer Out"]):
        print(f"{trade_type} {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp}");
    else:
        print(f"{trade_type} not yet recognized")
        raise f"{trade_type} not yet recognized"

    return this_acb

#go through all the tokens, calculate the ACB
all_coins = df.Currency.unique()

for token_name in all_coins:
    token_txs = tax_df[tax_df.Currency == token_name]
    #income_tx = df[df.Currency == token_name & df["Trade Type"] == ]
    curr_acb = None
    for index, row in token_txs.iterrows():
        curr_acb = calc_acb(curr_acb, row)

C:\Users\naaht\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Buy 0.0028606 Bitcoin (BTC) @ 17347.6081 on 2018-01-01 01:46:00
Selling 0.00181653 Bitcoin (BTC) @ 20433.41386 on 2018-01-06 03:42:00 gained -12.506658451754198
Buy 0.00251394 Bitcoin (BTC) @ 18046.52882 on 2018-01-10 11:15:00
Selling 0.00271777 Bitcoin (BTC) @ 14757.94851 on 2018-01-16 10:50:00 gained -8.9376049091087
Selling 0.00486212 Bitcoin (BTC) @ 14782.79341 on 2018-01-16 10:51:00 gained 71.8757154946292
Buy 0.00653363 Bitcoin (BTC) @ 14582.91287 on 2018-01-18 10:58:00
Buy 0.00243304 Bitcoin (BTC) @ 14132.2314 on 2018-01-19 04:54:00
Selling 0.00586462 Bitcoin (BTC) @ 15789.04095 on 2018-01-20 04:50:00 gained -2.682631678629093
Mining 0.00268294 Bitcoin (BTC) @ 13087.45536 on 2018-01-22 13:47:00
Selling 0.00371236 Bitcoin (BTC) @ 10955.99929 on 2018-02-01 23:14:00 gained 0.0
Mining 0.00458053 Bitcoin (BTC) @ 13898.20579 on 2018-02-21 06:11:00
Mining 0.00222837 Bitcoin (BTC) @ 13398.58051 on 2018-02-21 13:06:00
Transfer Out 0.00302853 Bitcoin (BTC) @ 13661.89951 on 2018-03-06 14:2

TypeError: exceptions must derive from BaseException

KeyError: "['Buy', 'Sell', 'Unknown Acquisition', 'Transfer In', 'Unknown Disposal', 'Transfer Out', 'Airdrop', nan, 'Fiat Deposit', 'Staking', 'Staking Deposit', 'Staking Withdrawal', 'Interest', 'Cashback', 'Failed Out', 'Income', 'Borrow', 'Burn', 'Mint', 'Loan Repayment', 'Fiat Withdrawal', 'Ignore In', 'Royalties', 'Unknown'] not in index"

In [ ]:
all_coins = df.Currency.unique()

df[""]